In [1]:
%%html
<style type='text/css'>
.CodeMirror{
font-size: 22px;
</style>

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(False)
gpus = tf.config.list_physical_devices('GPU')

2023-03-22 23:44:30.766923: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 23:44:30.900795: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-22 23:44:31.556015: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-22 23:44:31.556078: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

Num GPUs Available:  1


In [5]:
from sklearn.decomposition import PCA
import numpy as np
import tensorflow as tf

# Load MNIST data.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocessing: normalize pixel values to be between 0 and 1.
x_train = x_train / 255.0
x_test = x_test / 255.0

# Shuffle training data.
shuffle_index = np.random.permutation(len(x_train))
x_train, y_train = x_train[shuffle_index], y_train[shuffle_index]

# Convert target labels to binary classification (digit < 5 or digit >= 5).
y_train = (y_train < 5)
y_test = (y_test < 5)

# Convert labels to one-hot encoding.
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

# Track the data type of x_train.
dataType = x_train.dtype
print(f"Data type: {dataType}")

# Track the data type of y_test.
labelType = y_test.dtype
print(f"Data type: {labelType}")

# Print the one-hot encoded label for the first training example.
print(y_train[0])

# Instantiate the training dataset.
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

# Perform PCA to reduce dimensionality of x_train.
pca = PCA(n_components=0.7)  # retain 95% of variance
x_train_reduced = pca.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_reduced = pca.transform(x_test.reshape(x_test.shape[0], -1))

# Print the dimensions of the original and reduced datasets.
print("Original dimensions:", x_train.shape)
print("Reduced dimensions:", x_train_reduced.shape)


Data type: float64
Data type: float32
[1. 0.]
Original dimensions: (60000, 784)
Reduced dimensions: (60000, 26)


In [6]:
from tensorflow import keras
inputs = keras.Input(shape=(26,), name="digits")
x = keras.layers.Dense(32, activation="relu", kernel_initializer="uniform",bias_initializer="uniform")(inputs)
# x = keras.layers.Dense(128, activation="relu", kernel_initializer="uniform",bias_initializer="uniform")(x)
outputs = keras.layers.Dense(2, name="predictions",kernel_initializer="uniform",bias_initializer="uniform")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

2023-03-22 23:44:35.617801: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 23:44:36.041430: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22288 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:51:00.0, compute capability: 8.6


In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 digits (InputLayer)         [(None, 26)]              0         
                                                                 
 dense (Dense)               (None, 32)                864       
                                                                 
 predictions (Dense)         (None, 2)                 66        
                                                                 
Total params: 930
Trainable params: 930
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
# Instantiate a loss function.
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn,  metrics=["accuracy"])

In [11]:
history = model.fit(x_train_reduced, y_train, validation_data=(x_test_reduced, y_test), batch_size=128, epochs=1000, verbose=2)


Epoch 1/1000


2023-03-22 14:35:48.750847: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:219] failed to create cublas handle: cublas error
2023-03-22 14:35:48.750898: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:221] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2023-03-22 14:35:48.750926: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at matmul_op_impl.h:622 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support


InternalError: Graph execution error:

Detected at node 'model_1/dense_1/MatMul' defined at (most recent call last):
    File "/home/shiqi/anaconda3/envs/tensor-koopman/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/shiqi/anaconda3/envs/tensor-koopman/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/shiqi/anaconda3/envs/tensor-koopman/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/shiqi/anaconda3/envs/tensor-koopman/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/shiqi/anaconda3/envs/tensor-koopman/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 728, in start
      self.io_loop.start()
    File "/home/shiqi/anaconda3/envs/tensor-koopman/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in st

In [67]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(False)
gpus = tf.config.list_physical_devices('GPU')

from sklearn.decomposition import PCA
import numpy as np
import tensorflow as tf

# Load MNIST data.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocessing: normalize pixel values to be between 0 and 1.
x_train = x_train / 255.0
x_test = x_test / 255.0

# Shuffle training data.
shuffle_index = np.random.permutation(len(x_train))
x_train, y_train = x_train[shuffle_index], y_train[shuffle_index]

# Convert target labels to binary classification (digit < 5 or digit >= 5).
y_train = (y_train < 5)
y_test = (y_test < 5)

# Convert labels to one-hot encoding.
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

# Track the data type of x_train.
dataType = x_train.dtype
print(f"Data type: {dataType}")

# Track the data type of y_test.
labelType = y_test.dtype
print(f"Data type: {labelType}")

# Print the one-hot encoded label for the first training example.
print(y_train[0])

# Instantiate the training dataset.
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

# Perform PCA to reduce dimensionality of x_train.
pca = PCA(n_components=0.7)  # retain 95% of variance
x_train_reduced = pca.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_reduced = pca.transform(x_test.reshape(x_test.shape[0], -1))

# Print the dimensions of the original and reduced datasets.
print("Original dimensions:", x_train.shape)
print("Reduced dimensions:", x_train_reduced.shape)

node_set = [16, 32, 64, 128, 256, 1024]


from tensorflow import keras

for node_num in node_set:
    for training_id in range(10):
        inputs = keras.Input(shape=(26,), name="digits")
        x = keras.layers.Dense(node_num, activation="relu", kernel_initializer="uniform",bias_initializer="uniform")(inputs)
        outputs = keras.layers.Dense(2, name="predictions",kernel_initializer="uniform",bias_initializer="uniform")(x)
        model = keras.Model(inputs=inputs, outputs=outputs)
        
        # Instantiate an optimizer.
        optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
        # Instantiate a loss function.
        loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
        # Compile the model
        model.compile(optimizer=optimizer, loss=loss_fn,  metrics=["accuracy"])        
        
        weight_record = []
        print("Start training at width %d, id %d" % (node_num, training_id))
        for epoch in range(200):
            history = model.fit(x_train_reduced, y_train, validation_data=(x_test_reduced, y_test), batch_size=128, epochs=1, verbose=2)
            weight_record.append(model.trainable_weights)
        np.save('learn_output/weight_%d_%d.npy' % (node_num, training_id), weight_record)
        del weight_record
        gc.collect()

Num GPUs Available:  1
Data type: float64
Data type: float32
[1. 0.]
Original dimensions: (60000, 784)
Reduced dimensions: (60000, 26)
Start training at width 16, id 0
 14/469 [..............................] - ETA: 1s - loss: 0.2312 - accuracy: 0.9129

KeyboardInterrupt: 